### Arduino - Colour Sensor Readings

In [ ]:
import serial
import time
import numpy as np
import matplotlib.pylab as plt

In [ ]:
start_time = time.time()
curr_time = time.time() - start_time
## print(curr_time)
connection_time = 0

data = np.array([np.zeros(6)])
timer = []

# to open the serial port

ser = serial.Serial('COM5', 9600)

while curr_time < 2:
    
    # read line from serial port
    line = ser.readline().decode().strip()
    ## print(line)

    if connection_time == 0:
        # we need to evalute the time it takes to connect the sensor
        connection_time = time.time() - start_time;
        ## print(connection_time)
        
    # calculate correct current time
    curr_time = time.time() - start_time - connection_time
    timer.append(curr_time)
    ## print(curr_time)

    # store reading values
    values = np.array(line.split(';')[:-1], dtype=int)
    
    if len(values) == 6:
        data = np.vstack((data,values))
    
if curr_time >= 1:
    # close serial reading after 5 seconds
    data = data[1:]
    timer = timer[1:]
    ser.close()

print(data)

fig, ax = plt.subplots(3,figsize = (7,10))
fig.subplots_adjust(hspace=0.5)
colors = ['red','lime','cyan']
rgb = []
print(np.median(data[:,2:5],axis=0))

for i in range(3):
    
    mean = np.mean(data[:,2+i])
    median = np.median(data[:,2+i])
    
    ax[i].hist(data[:,2+i],bins = 10,edgecolor = 'black',color=colors[i],alpha = 0.5)
    
    ax[i].axvline(mean,0,1, color = 'black',linewidth='2',label='mean')
    
    ax[i].axvline(median,0,1, color = 'magenta',linewidth='2',label='median')
    
    ax[i].legend()
    ax[i].set(xlabel='Measurment of the discrete Stimulus',ylabel='Count')
    ax[i].grid()

    rgb.append(median)
    
print(rgb)

### PyQt5 Appliction

In [2]:
import sys
from PyQt5.QtCore import Qt,QTimer, QThread, pyqtSignal, pyqtSlot
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLineEdit, QProgressBar, QLabel, QFileDialog, QVBoxLayout
from PyQt5.QtGui import QFont, QPainter, QColor
import serial
import pandas as pd
import numpy as np
import time
from scipy.optimize import curve_fit
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

class ColorSquare(QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.color = QColor(0, 0, 0)

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.fillRect(self.rect(), self.color)

class AcquisitionThread(QThread):
    data_acquired = pyqtSignal(np.ndarray)
    progress_updated = pyqtSignal(int)

    def __init__(self, serial_port, serial_baudrate):
        super().__init__()
        self.serial_port = serial_port
        self.serial_baudrate = serial_baudrate
        self.running = False

    def run(self):
        self.running = True
        data = np.array([np.zeros(6)])  # Clear previous data
        start_time = time.time()
        curr_time = time.time() - start_time
        connection_time = 0

        try:
            ser = serial.Serial(self.serial_port, self.serial_baudrate)
            while self.running and curr_time < 1:
                line = ser.readline().decode().strip()

                if connection_time == 0:
                    connection_time = time.time() - start_time

                curr_time = time.time() - start_time - connection_time
                self.progress_updated.emit(int(curr_time * 1000))  # Update progress bar

                values = np.array(line.split(';')[:-1], dtype=int)
                if len(values) == 6:
                    data = np.vstack((data, values))

            self.progress_updated.emit(1000)
            ser.close()
            
        except serial.SerialException as e:
            print(f"Serial error: {e}")
        except Exception as e:
            print(f"Error: {e}")

        self.data_acquired.emit(data)
        self.running = False

    def stop(self):
        self.running = False
        self.wait()

class PlotCanvas(FigureCanvas):
    def __init__(self, parent=None):
        fig = Figure()
        self.axes = fig.add_subplot(3, 2, (2,4))
        super().__init__(fig)

        self.setParent(parent)
        self.lower()

    def plot_rawdata(self, x_data, y_data):
        self.axes.clear()
        self.axes.scatter(x_data[:, 0], y_data[:, 0], c='r', label='Red')
        self.axes.scatter(x_data[:, 1], y_data[:, 1], c='g', label='Green')
        self.axes.scatter(x_data[:, 2], y_data[:, 2], c='b', label='Blue')
        self.axes.legend()
        self.axes.set_title("Raw Data Plot")
        self.axes.set_xlabel('Input RGB')
        self.axes.set_ylabel('Measured RGB')
        self.axes.grid(True)
        self.draw()

    def plot_tf3(self, x, y, a, coef):

        if a == 'Red':
            self.axes.clear()
            
        xx = np.linspace(np.min(x),np.max(x),500)
        self.axes.scatter(x,y,label='Raw {} Data '.format(a),c = a)
        self.axes.plot(xx,xx**3*coef[0] + xx**2*coef[1]+ xx*coef[2] + coef[3],label='{} Transfer Function'.format(a),c = a)
        self.axes.legend()
        self.axes.set_title("Transfer Function Plot")
        self.axes.set_xlabel('Input RGB')
        self.axes.set_ylabel('Measured RGB')
        self.axes.grid(True)
        self.draw()

    def plot_tf2(self, x, y, a, coef):

        if a == 'Red':
            self.axes.clear()
            
        xx = np.linspace(np.min(x),np.max(x),500)
        self.axes.scatter(x,y,label='Raw {} Data '.format(a),c = a)
        self.axes.plot(xx,xx**2*coef[0] + xx*coef[1]+ coef[2],label='{} Transfer Function'.format(a),c = a)
        self.axes.legend()
        self.axes.set_title("Transfer Function Plot")
        self.axes.set_xlabel('Input RGB')
        self.axes.set_ylabel('Measured RGB')
        self.axes.grid(True)
        self.draw()

    def plot_tf1(self, x, y, a, coef):

        if a == 'Red':
            self.axes.clear()
            
        xx = np.linspace(np.min(x),np.max(x),500)
        self.axes.scatter(x,y,label='Raw {} Data '.format(a),c = a)
        self.axes.plot(xx,xx*coef[0] + coef[1],label='{} Transfer Function'.format(a),c = a)
        self.axes.legend()
        self.axes.set_title("Transfer Function Plot")
        self.axes.set_xlabel('Input RGB')
        self.axes.set_ylabel('Measured RGB')
        self.axes.grid(True)
        self.draw()
    
    def plot_sense(self, x, y, a):

        if a == 'Red':
            self.axes.clear()

        self.axes.plot(x,y,label='{} Sensitivity Function'.format(a),c = a)
        self.axes.legend()
        self.axes.set_title("Sensitivity Plot")
        self.axes.set_xlabel('Stimulus')
        self.axes.set_ylabel('Sensitivity')
        self.axes.grid(True)
        self.draw()


class TCS_Calibration_APP(QWidget):
    def __init__(self):
        super().__init__()

        #################### Variables #####################
        self.serial_port = 'COM5'
        self.serial_baudrate = 9600
        self.data = np.array([np.zeros(6)])
        
        self.data_rgb_measured = np.array([np.zeros(3)])
        self.data_rgb_input = np.array([np.zeros(3)])
        self.data_uncertainty = np.array([np.zeros(3)])
        
        self.rgb = np.zeros(3)
        self.measurement_counter = 0
        self.total_measurements = 12 #default

        ####################################################
        
        self.setStyleSheet("background-color: white;")

        self.setWindowTitle('Data Acquisition')
        self.setGeometry(700, 50, 1200, 900)

        font1 = QFont()
        font1.setFamily("Arial")
        font1.setPointSize(12)  
        font1.setBold(True) 
        
        self.textLabel1 = QLabel("Press to start calibration", self)
        self.textLabel1.setGeometry(10,0,400,50)
        self.textLabel1.setFont(font1)

        self.start_button_measure = QPushButton('Start', self)
        self.start_button_measure.setGeometry(375,0,100,50)
        self.start_button_measure.clicked.connect(self.start_measurements)
        self.start_button_measure.setStyleSheet("""
            QPushButton {
                background-color: #8a99ff;  
                color: black;              /* White text */
                border: 1px solid #8a99ff;   /* Black border */
                font-weight: bold;                                 
            }
            QPushButton:hover {
                background-color: #5e72f7;  /* Darker blue on hover */
            }
            QPushButton:pressed {
                background-color: #3c55fa;  /* Even darker blue on press */
            }
        """)
        
        self.progress_bar = QProgressBar(self)
        self.progress_bar.setGeometry(300,75,200,25)
        self.progress_bar.setMaximum(1000)  # Max value corresponds to 1 second

        font2 = QFont()
        font2.setFamily("Arial")
        font2.setPointSize(12)  
        
        self.textLabel2 = QLabel("Chose your RGB colour", self)
        self.textLabel2.setGeometry(10,100,400,50)
        self.textLabel2.setFont(font2)

        self.textLabel3 = QLabel("Measured RGB colour", self)
        self.textLabel3.setGeometry(290,100,200,50)
        self.textLabel3.setFont(font2)

        self.textLabel4 = QLabel("Plotting Buttons", self)
        self.textLabel4.setGeometry(290,400,200,50)
        self.textLabel4.setFont(font2)

        self.button_tf = QPushButton('Transfer Function', self)
        self.button_tf.setGeometry(290,500,200,45)
        self.button_tf.clicked.connect(self.tf_calc_plot)
        self.button_tf.setStyleSheet("""
            QPushButton {
                background-color: #8a99ff;  
                color: black;              /* White text */
                border: 1px solid #8a99ff;   /* Black border */
                font-weight: bold;                                 
            }
            QPushButton:hover {
                background-color: #5e72f7;  /* Darker blue on hover */
            }
            QPushButton:pressed {
                background-color: #3c55fa;  /* Even darker blue on press */
            }
        """)

        self.button_tf1 = QPushButton('Linear', self)
        self.button_tf1.setGeometry(500,500,100,20)
        self.button_tf1.clicked.connect(self.trigger_linear_fit)
        self.button_tf1.setStyleSheet("""
            QPushButton {
                background-color: #8a99ff;  
                color: black;              /* White text */
                border: 1px solid #8a99ff;   /* Black border */
                font-weight: bold;                                 
            }
            QPushButton:hover {
                background-color: #5e72f7;  /* Darker blue on hover */
            }
            QPushButton:pressed {
                background-color: #3c55fa;  /* Even darker blue on press */
            }
        """)

        self.button_tf3 = QPushButton('Polynomial', self)
        self.button_tf3.setGeometry(500,525,100,20)
        self.button_tf3.clicked.connect(self.trigger_polynomial_fit)
        self.button_tf3.setStyleSheet("""
            QPushButton {
                background-color: #8a99ff;  
                color: black;              /* White text */
                border: 1px solid #8a99ff;   /* Black border */
                font-weight: bold;                                 
            }
            QPushButton:hover {
                background-color: #5e72f7;  /* Darker blue on hover */
            }
            QPushButton:pressed {
                background-color: #3c55fa;  /* Even darker blue on press */
            }
        """)


        self.button_rawdata = QPushButton('Raw Data Plot', self)
        self.button_rawdata.setGeometry(290,450,200,45)
        self.button_rawdata.clicked.connect(self.plotting_data)
        self.button_rawdata.setStyleSheet("""
            QPushButton {
                background-color: #8a99ff;  
                color: black;              /* White text */
                border: 1px solid #8a99ff;   /* Black border */
                font-weight: bold;                                 
            }
            QPushButton:hover {
                background-color: #5e72f7;  /* Darker blue on hover */
            }
            QPushButton:pressed {
                background-color: #3c55fa;  /* Even darker blue on press */
            }
        """)

        self.button_rawdata = QPushButton('Sensitivity Plot', self)
        self.button_rawdata.setGeometry(290,550,200,45)
        self.button_rawdata.clicked.connect(self.sensitivity_plot_calculations)
        self.button_rawdata.setStyleSheet("""
            QPushButton {
                background-color: #8a99ff;  
                color: black;              /* White text */
                border: 1px solid #8a99ff;   /* Black border */
                font-weight: bold;                                 
            }
            QPushButton:hover {
                background-color: #5e72f7;  /* Darker blue on hover */
            }
            QPushButton:pressed {
                background-color: #3c55fa;  /* Even darker blue on press */
            }
        """)


        self.red_input = QLineEdit(self)
        self.red_input.setPlaceholderText('Red (0-255)')
        self.red_input.setGeometry(55,150,90,25)
        self.red_input.textChanged.connect(self.updateColor)

        self.green_input = QLineEdit(self)
        self.green_input.setPlaceholderText('Green (0-255)')
        self.green_input.setGeometry(55,180,90,25)
        self.green_input.textChanged.connect(self.updateColor)

        self.blue_input = QLineEdit(self)
        self.blue_input.setPlaceholderText('Blue (0-255)')
        self.blue_input.setGeometry(55,210,90,25)
        self.blue_input.textChanged.connect(self.updateColor)

        self.numbermeas_input = QLineEdit(self)
        self.numbermeas_input.setPlaceholderText('Nº colours (int)')
        self.numbermeas_input.setGeometry(10,50,125,30)
      

        self.color_square = ColorSquare(self)
        self.color_square.setGeometry(25,250,150,150)

        font3 = QFont()
        font3.setFamily("Arial")
        font3.setPointSize(9)

        self.red_output = QLabel("Red: {}".format(int(self.rgb[0])), self)
        self.red_output.setGeometry(300,150,90,25)
        self.red_output.setFont(font3)
        self.green_output = QLabel("Green: {}".format(int(self.rgb[1])), self)
        self.green_output.setGeometry(300,180,90,25)
        self.green_output.setFont(font3)
        self.blue_output = QLabel("Blue: {}".format(int(self.rgb[2])), self)
        self.blue_output.setGeometry(300,210,90,25)
        self.blue_output.setFont(font3)

        self.red_tf = QLabel("Red Transfer Function:", self)
        self.red_tf.setGeometry(600,650,550,25)
        self.red_tf.setFont(font3)
        self.green_tf = QLabel("Green Transfer Function:", self)
        self.green_tf.setGeometry(600,680,550,25)
        self.green_tf.setFont(font3)
        self.blue_tf = QLabel("Blue Transfer Function:", self)
        self.blue_tf.setGeometry(600,710,550,25)
        self.blue_tf.setFont(font3)

        self.color_square_measured = ColorSquare(self)
        self.color_square_measured.setGeometry(300,250,150,150)

        # File importing

        self.file_path_label = QLabel("No file selected.",self)
        self.file_path_label.setGeometry(55,420,100,25)

        self.load_button = QPushButton("Load Excel File",self)
        self.load_button.setGeometry(50,450,100,40)
        self.load_button.clicked.connect(self.load_excel_file)
        self.load_button.setStyleSheet("""
            QPushButton {
                background-color: #8db87f;  
                color: black;              /* White text */
                border: 1px solid #8db87f;   /* Black border */
                font-weight: bold;                                 
            }
            QPushButton:hover {
                background-color: #74b85e;  /* Darker blue on hover */
            }
            QPushButton:pressed {
                background-color: #538542;  /* Even darker blue on press */
            }
        """)

        # Timer for periodic measurements
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.acquisition_thread)

        # Plot area
        self.plot_canvas = PlotCanvas(self)
        layout = QVBoxLayout()
        layout.addWidget(self.plot_canvas)
        self.setLayout(layout)

        ######################################################

        self.textLabel5 = QLabel("Press to measure calibrated colour", self)
        self.textLabel5.setGeometry(10,650,400,50)
        self.textLabel5.setFont(font1)

        self.start_guess = QPushButton('Measure', self)
        self.start_guess.setGeometry(350,650,100,50)
        self.start_guess.clicked.connect(self.start_measurements_two)
        self.start_guess.setStyleSheet("""
            QPushButton {
                background-color: #ffb732;  
                color: black;              /* White text */
                border: 1px solid #ffb732;   /* Black border */
                font-weight: bold;                                 
            }
            QPushButton:hover {
                background-color: #FFA500;  /* Darker blue on hover */
            }
            QPushButton:pressed {
                background-color: #e59400;  /* Even darker blue on press */
            }
        """)


    def updateColor(self):
        try:
            # Get RGB values from input fields
            red = int(self.red_input.text() or 0)
            green = int(self.green_input.text() or 0)
            blue = int(self.blue_input.text() or 0)

            # Validate RGB values
            if 0 <= red <= 255 and 0 <= green <= 255 and 0 <= blue <= 255:
                self.color_square.color = QColor(red, green, blue)
                self.color_square.update()  # Update the color square
            else:
                raise ValueError("RGB values must be between 0 and 255")
        except ValueError as e:
            print("Invalid input:", e)

    def start_measurements(self):
        self.start_button_measure.setEnabled(False)  # Disable the start button
        self.measurement_counter = 0
        self.data_rgb_measured = np.array([np.zeros(3)])

        try:
            self.total_measurements = int(self.numbermeas_input.text())
        except:
            self.total_measurements = 12 # default

        self.timer.start(5000)  # start a timer to trigger the function associeated to timer every 5000 milliseconds (5 seconds)

    def start_measurements_two(self):

        data = np.array([np.zeros(6)])  # Clear previous data
        start_time = time.time()
        curr_time = time.time() - start_time
        connection_time = 0
        self.progress_bar.setValue(0)  # Update progress bar
        ser = serial.Serial(self.serial_port, self.serial_baudrate)

        while curr_time < 1:
            line = ser.readline().decode().strip()

            if connection_time == 0:
                connection_time = time.time() - start_time

            curr_time = time.time() - start_time - connection_time
            self.progress_bar.setValue(int(curr_time * 1000))  # Update progress bar

            values = np.array(line.split(';')[:-1], dtype=int)
            if len(values) == 6:
                data = np.vstack((data, values))

        self.progress_bar.setValue(1000) # Update progress bar
        ser.close()

        rgb = np.mean(data[1:, 2:5] / data[1:, 5][:, np.newaxis], axis=0) * 255
        uncertainty_a = np.std(data[1:, 2:5] / data[1:, 5][:, np.newaxis] * 255, axis=0)
        uncertainty_b = np.ones(3) * 255/21504  # depends on integration time and gain 
        uncertainty = np.sqrt(uncertainty_a**2 + uncertainty_b**2)

        ## Use calibration function to get expected rgb values by inverting transfer function

            
        for i in range(3):

            if (self.poly_fit  == True):

                a = self.coef3[i+1,0]
                b = self.coef3[i+1,1]
                c = self.coef3[i+1,2]
                d = self.coef3[i+1,3] - rgb[i]


                roots = np.roots([a,b,c,d])
                root = roots[np.argmin(roots - rgb[i])]

                if root <= 0: 
                    rgb[i] = 0
                elif root >= 255:
                    rgb[i] = 255
                else:
                    rgb[i] = root

            elif (self.lin_fit == True):

                a = self.coef1[i+1,0]
                b = self.coef1[i+1,1]

                root = (rgb[i] - b)/a

                if root <= 0: 
                    rgb[i] = 0
                elif root >= 255:
                    rgb[i] = 255
                else:
                    rgb[i] = root

        self.color_square_measured.color = QColor(int(rgb[0]), int(rgb[1]), int(rgb[2]))
        self.red_output.setText("Red: {}".format(int(rgb[0])))
        self.green_output.setText("Green: {}".format(int(rgb[1])))
        self.blue_output.setText("Blue: {}".format(int(rgb[2])))
        self.color_square_measured.update() 


    def acquisition_thread(self):

        self.measurement_counter += 1

        if self.measurement_counter <= self.total_measurements:
            print(f"Measurement {self.measurement_counter}/{self.total_measurements}")
            self.progress_bar.setValue(0)  # Reset the progress bar
            self.acquisition_thread_instance = AcquisitionThread(self.serial_port, self.serial_baudrate)
            self.acquisition_thread_instance.data_acquired.connect(self.data_treatment)
            self.acquisition_thread_instance.progress_updated.connect(self.update_progress_bar)
            self.acquisition_thread_instance.start()
        else:
            self.timer.stop()
            self.start_button_measure.setEnabled(True)  # Re-enable the start button

    def load_excel_file(self):
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Open Excel File", "", "Excel Files (*.xlsx *.xls)")
        if file_path:
            self.file_path_label.setText("File Loaded")
            # Read the Excel file using pandas
            self.data_rgb_input = pd.read_excel(file_path).values
            print(self.data_rgb_input)
            
    @pyqtSlot(int)
    def update_progress_bar(self, value):
        self.progress_bar.setValue(value)

    @pyqtSlot(np.ndarray)
    def data_treatment(self, data):
        self.data = data
        # print(self.data)
        
        # For any color measurement
        self.rgb = np.mean(self.data[1:, 2:5] / self.data[1:, 5][:, np.newaxis], axis=0) * 255
        self.uncertainty_a = np.std(self.data[1:, 2:5] / self.data[1:, 5][:, np.newaxis] * 255, axis=0)
        self.uncertainty_b = np.ones(3) * 255/21504  # depends on integration time and gain 
        self.uncertainty = np.sqrt(self.uncertainty_a**2 + self.uncertainty_b**2)

        self.color_square_measured.color = QColor(int(self.rgb[0]), int(self.rgb[1]), int(self.rgb[2]))
        self.color_square_measured.update() 

        self.color_square.color = QColor(int(self.data_rgb_input[self.measurement_counter - 1, 0]), int(self.data_rgb_input[self.measurement_counter - 1, 1]), int(self.data_rgb_input[self.measurement_counter - 1, 2]))
        self.color_square.update() 
        
        self.red_output.setText("Red: {}".format(int(self.rgb[0])))
        self.green_output.setText("Green: {}".format(int(self.rgb[1])))
        self.blue_output.setText("Blue: {}".format(int(self.rgb[2])))

        # Store measurements
        self.data_rgb_measured = np.vstack((self.data_rgb_measured, self.rgb))
        self.data_uncertainty = np.vstack((self.data_uncertainty, self.uncertainty))

        print(self.data_rgb_measured[1:])
        print(self.data_rgb_input[:self.measurement_counter])
        print(self.data_uncertainty[1:])

        # Update plot
        self.plot_canvas.plot_rawdata(self.data_rgb_input[:self.measurement_counter], self.data_rgb_measured[1:])



    def plotting_data(self):

        if self.measurement_counter >= self.total_measurements:

            self.plot_canvas.plot_rawdata(self.data_rgb_input[:self.measurement_counter], self.data_rgb_measured[1:])

    def tf_calc_plot(self):

        def f3(x,a,b,c,d):
            return a*x**3 + b*x**2 + c*x + d
        
        def f2(x,a,b,c):
            return a*x**2 + b*x + c

        def f1(x,a,b):
            return a*x + b
        
        if self.measurement_counter >= self.total_measurements:
            a = ["Red","Green","Blue"]
            self.coef1 = np.zeros(2)
            self.coef3 = np.zeros(4)

            for i in range(3):

                stimulus = np.sort(self.data_rgb_input[:,i])
                measurement = self.data_rgb_measured[1:,i][np.argsort(self.data_rgb_input[:,i])]

                c1, pcov1 = curve_fit(f1,stimulus, measurement)
                c2, pcov2 = curve_fit(f2,stimulus, measurement)
                c3, pcov3 = curve_fit(f3,stimulus, measurement)
                #print(coef)
                
                #self.mse1 = 1/len(stimulus) * np.sum(( measurement - (coef1[0] * stimulus + coef1[1]) )**2)
                #self.mse2 = 1/len(stimulus) * np.sum(( measurement - (coef2[0] * stimulus ** 2 + coef2[1] *  stimulus + coef2[2]) )**2)
                #self.mse3 = 1/len(stimulus) * np.sum(( measurement - (coef3[0] * stimulus ** 3 + coef3[1] *  stimulus ** 2 + coef3[2] * stimulus + coef3[3]) )**2)
                
                # by default, clicking transfer function will plot 3rd degree polynomial
                self.plot_canvas.plot_tf3(stimulus, measurement,a[i],c3)
                self.lin_fit = False
                self.poly_fit = True

                # store coeficients
                self.coef3 = np.vstack((self.coef3, c3))
                self.coef1 = np.vstack((self.coef1, c1))

                #self.plot_canvas.plot_tf1(stimulus, measurement,a[i],coef1)
                #self.coef = np.vstack((self.coef, np.append(coef1,[0,0])))

    def trigger_linear_fit(self):

        self.lin_fit = True
        self.poly_fit = False

        self.red_tf.setText("Red Transfer Function:  y = {} x + {}".format(np.round(self.coef1[1,0],5),np.round(self.coef1[1,1],5)))
        self.green_tf.setText("Green Transfer Function:  y = {} x + {}".format(np.round(self.coef1[2,0],5),np.round(self.coef1[2,1],5)))
        self.blue_tf.setText("Blue Transfer Function:  y = {} x + {}".format(np.round(self.coef1[3,0],5),np.round(self.coef1[3,1],5)))

        a = ["Red","Green","Blue"]
        for i in range(3):
                stimulus = np.sort(self.data_rgb_input[:,i])
                measurement = self.data_rgb_measured[1:,i][np.argsort(self.data_rgb_input[:,i])]
                self.plot_canvas.plot_tf1(stimulus, measurement,a[i],self.coef1[i+1,:])

    def trigger_polynomial_fit(self):

        self.lin_fit = False
        self.poly_fit = True

        self.red_tf.setText("Red Transfer Function:  y = {} x^3 + {} x^2 + {} x + {}".format(np.round(self.coef3[1,0],5),np.round(self.coef3[1,1],5),np.round(self.coef3[1,2],5),np.round(self.coef3[1,3],5)))
        self.green_tf.setText("Green Transfer Function:  y = {} x^3 + {} x^2 + {} x + {}".format(np.round(self.coef3[2,0],5),np.round(self.coef3[2,1],5),np.round(self.coef3[2,2],5),np.round(self.coef3[2,3],5)))
        self.blue_tf.setText("Blue Transfer Function:  y = {} x^3 + {} x^2 + {} x + {}".format(np.round(self.coef3[3,0],5),np.round(self.coef3[3,1],5),np.round(self.coef3[3,2],5),np.round(self.coef3[3,3],5)))

        a = ["Red","Green","Blue"]
        for i in range(3):
                stimulus = np.sort(self.data_rgb_input[:,i])
                measurement = self.data_rgb_measured[1:,i][np.argsort(self.data_rgb_input[:,i])]
                self.plot_canvas.plot_tf3(stimulus, measurement,a[i],self.coef3[i+1,:])



    def sensitivity_plot_calculations(self):

        colours_array = ["Red","Green","Blue"]

        for i in range(3):

            x = np.linspace(np.min(self.data_rgb_measured[1:,i]),np.max(self.data_rgb_measured[1:,i]),500)
            h = 1e-5
            xx = x + h

            a = self.coef3[i+1,0]
            b = self.coef3[i+1,1]
            c = self.coef3[i+1,2]
            d = self.coef3[i+1,3]  
        
            sense = (a*xx**3 + b*xx**2 + c*xx + d - a*x**3 - b*x**2 - c*x - d)/h
         
            self.plot_canvas.plot_sense(x,sense,colours_array[i])

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = TCS_Calibration_APP()
    window.show()
    app.exec_()


[[ 10  10  10]
 [250 250 250]
 [156  39  31]
 [189  71  32]
 [242 216  46]
 [153 184  68]
 [ 21  94  54]
 [ 40 153 181]
 [ 24  29  77]
 [ 66  36  87]
 [170 125 201]
 [235 160 195]]
Measurement 1/12
[[90.82529559 89.70678461 64.47425435]]
[[10 10 10]]
[[0.03651165 0.02887874 0.02195127]]
Measurement 2/12
[[ 90.82529559  89.70678461  64.47425435]
 [255.         255.         193.37131076]]
[[ 10  10  10]
 [250 250 250]]
[[0.03651165 0.02887874 0.02195127]
 [0.01185826 0.01185826 0.03850413]]
Measurement 3/12
[[ 90.82529559  89.70678461  64.47425435]
 [255.         255.         193.37131076]
 [176.84032423  43.08522071  36.62014437]]
[[ 10  10  10]
 [250 250 250]
 [156  39  31]]
[[0.03651165 0.02887874 0.02195127]
 [0.01185826 0.01185826 0.03850413]
 [0.02152547 0.01710188 0.0147842 ]]
Measurement 4/12
[[ 90.82529559  89.70678461  64.47425435]
 [255.         255.         193.37131076]
 [176.84032423  43.08522071  36.62014437]
 [180.67555363  52.15012629  31.8980924 ]]
[[ 10  10  10]
 [250 

AttributeError: 'TCS_Calibration_APP' object has no attribute 'coef1'

: 

In [ ]:
import numpy as np
import matplotlib.pylab as plt
from scipy.interpolate import CubicSpline, interp1d
from scipy.optimize import curve_fit

def f(x, a,b,c,d):
    return a*x**3 + b*x**2 + c*x + d

y_y = np.array([[103.75039002,98.63657624,78.44929797],
 [255.,255.,255.],
 [187.06369552,53.34491425,48.99689533],
 [250.49386161,78.03920201,55.46107701],
 [255.,255.,109.35686384],
 [190.20647321,230.77950614,100.81891741],
 [74.65112616,127.96392976,74.12899669],
 [61.3546317,111.57435826,106.97335379],
 [86.96033967,88.67532091,130.03474233],
 [119.80142857,87.266235,108.89359878],
 [150.01883371,109.78376116,132.52790179],
 [255.,206.47600446,177.29282924]])

x = np.array([[10 , 10 , 10],
 [250, 250 ,250],
 [156 , 39 , 31],
 [189 , 71 , 32],
 [242 ,216 , 46],
 [153, 184 , 68],
 [ 21 , 94 , 54],
 [ 40 ,153 ,181],
 [ 24 , 29  ,77],
 [ 66 , 36  ,87],
 [170, 125 ,201],
 [235 ,160 ,195]])

y = np.array([[102.     ,     97.61947241,  75.62898292],
 [255.      ,   255.     ,    255.        ],
 [212.48814174,  59.43359375 , 53.82463728],
 [255.       ,   91.52204241 , 64.04645647],
 [255.       ,  255.        , 131.24720982],
 [255.       ,  255.        , 131.98242188],
 [ 85.03557478, 157.35909598 , 87.06333705],
 [ 71.48158482 ,136.20396205 ,130.15625   ],
 [ 83.12375992 , 86.46103628 ,129.12353233],
 [115.35278641 , 83.84787115 ,105.07057416],
 [170.62848772, 126.35567801, 151.15722656],
 [255.        , 255.  ,       225.11125837]])

a = ["Red","Green","Blue"]
print(np.sort(x[:,0]))
print(y[:,0][np.argsort(x[:,0])])

for i in range(3):

    stimulus = np.sort(x[:,i])
    measurement = y[:,i][np.argsort(x[:,i])]

    coef, pcov = curve_fit(f,stimulus, measurement)
    #print(coef)
    coef_sigma = np.sqrt(np.diag(pcov))
    #print(coef_sigma)

    #cs = interp1d(stimulus, measurement, kind = 'quadratic')
    xx = np.linspace(10,250,1000)

    plt.figure(1)
    plt.scatter(stimulus,measurement,label='Raw {} Data '.format(a[i]),c = a[i])
    plt.plot(xx,xx**3*coef[0] + xx**2*coef[1]+ xx*coef[2] + coef[3],label='{} Transfer Function'.format(a[i]),c = a[i])
    plt.legend()
    plt.grid()

    xx = np.linspace(10,250,12)

    residual = measurement - (xx**3*coef[0] + xx**2*coef[1]+ xx*coef[2] + coef[3])
    ssr = np.sum(residual**2)
    std = np.sqrt(ssr / (len(xx) - 2))

    print(std)

    plt.figure(2)
    plt.hlines(2*std,0,255,linestyles='dashed',color = a[i])
    plt.hlines(-2*std,0,255,linestyles='dashed',color = a[i])
    plt.hlines(0,0,255,color='black')
    plt.scatter(xx, residual, label = "Residual {} Data".format(a[i]), c = a[i])
    plt.grid()
    plt.legend()




In [ ]:
a = 1
b = 2
c = 3
d = 4

print(np.real(np.roots([a,b,0,0])))